In [44]:
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import cPickle as pickle
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet as wn
import json
from nltk.tokeniz import TweetTokenizer


In [45]:
wnl = WordNetLemmatizer()

In [46]:
with open('./data/imagenet_top1k_captions.pkl', 'rb') as f:
    captions = pickle.load(f)
with open('./data/top1k.pkl', 'rb') as f:
    top1k = pickle.load(f)    


In [64]:
?df.applymap
df = pd.DataFrame(top1k,columns=['wnid','word'])
df['captions'] = df.apply(lambda x: captions[x.wnid], axis=1)
df['num_captions'] = df.apply(lambda x: len(x.captions), axis=1)
df.head()

,wnid,word,captions,num_captions
0,n10582746,man,[a man in a suit and tie standing in a field ....,1418
1,n10787470,woman,"[two women are standing together with a tie .,...",1405
2,n04379243,table,[a living room with a wooden table and chairs ...,1945
3,n04334599,street,[a train traveling down tracks next to a fores...,1297
4,n00007846,person,"[a man and a woman are posing for a picture .,...",1242


In [23]:
errPerWord = {}
# from IPython.core.debugger import Tracer
# Tracer()()
for i, (k, w) in enumerate(top1k):
    wl = wnl.lemmatize(w)
    print w, wl, k
    errors = sum([1 for cap in captions[k] if wl not in map(wnl.lemmatize, nltk.word_tokenize(cap))])
    print errors, len(captions[k]), errors*100.0/len(captions[k])

man man
710 1418 50.0705218618
woman woman
523 1405 37.2241992883
table table
1219 1945 62.6735218509
street street
706 1297 54.433307633
person person
1209 1242 97.3429951691
field field
579 1289 44.918541505
top top
47 59 79.6610169492
tennis tennis
552 1837 30.0489929232
front front
13 13 100.0
room room
836 1326 63.0467571644
plate plate
1166 1183 98.5629754861
train train
262 1312 19.9695121951
dog dog
743 1603 46.3505926388
water water
608 759 80.1054018445
cat cat
392 1485 26.3973063973
baseball baseball
765 1851 41.3290113452
bathroom bathroom
257 1304 19.7085889571
sign sign
370 1201 30.8076602831
kitchen kitchen
305 1275 23.9215686275
food food
655 1272 51.4937106918
grass grass
1088 1274 85.4003139717
bus bus
189 1338 14.1255605381
pizza pizza
98 1289 7.60279286268
side side
1815 1835 98.9100817439
building building
561 1421 39.4792399719
bed bed
1038 1741 59.6209075244
snow snow
1014 1313 77.2277227723
ball ball
1207 1244 97.0257234727
beach beach
1095 1713 63.9229422067
bo

# Flexible Matching
- Direct hyponyms
- full hyponyms
- Vector similarity 

In [26]:
# Direct hyponyms
person = wn.synsets('person')[0]

def direct_hyponyms(synset): 
    direct_hypo = [] 
    for ss in synset.hyponyms():
        direct_hypo += ss.lemma_names()
    direct_hypo =  sorted(direct_hypo) 
    return direct_hypo


In [28]:
# Full hyponyms
def full_hyponyms(ss):
    full_hypo = []
    q = [ss]
    while len(q) > 0:
        a = q.pop(0)
        q += a.hyponyms()
        full_hypo += [l.replace('_',' ') for l in a.lemma_names()]
    return full_hypo

full_hypo = full_hyponyms(person)
print len(full_hypo)
print full_hypo.index('person')
print sorted(full_hypo)
print full_hypo[:5]

13235
0
[u'1st lieutenant', u'2nd lieutenant', u'AWOL', u'Abenaki', u'Abenaki', u'Abkhas', u'Abkhasian', u'Abkhaz', u'Abkhazian', u'Abnaki', u'Abnaki', u'Abo', u'Aboriginal', u'Aborigine', u'Acadian', u'Achaean', u'Achaian', u'Achomawi', u'Achomawi', u'Adventist', u'Aeolian', u'Afghan', u'Afghan', u'Afghanistani', u'Afghanistani', u'African', u'African American', u'African-American', u'Afrikander', u'Afrikaner', u'Afro-American', u'Agriculture Secretary', u"Akwa'ala", u"Akwa'ala", u'Alabama', u'Alabama', u'Alabaman', u'Alabamian', u'Alaska Native', u'Alaskan', u'Alaskan Native', u'Albanian', u'Aleut', u'Aleut', u'Aleutian', u'Aleutian', u'Alexandrian', u'Algerian', u'Algonkian', u'Algonkian', u'Algonkin', u'Algonkin', u'Algonquian', u'Algonquian', u'Algonquin', u'Algonquin', u'Almoravid', u'Alsatian', u'Altaic', u'Altaic', u'American', u'American', u'American Indian', u'American Indian', u'American Revolutionary leader', u'American Revolutionary leader', u'Amerindian', u'Amerindian', u

In [31]:
top1k[0]


(u'n10582746', 'man')

In [32]:
# Get Direct hypo
from collections import Counter
errPerWord = {}
for i,(k,w) in enumerate(top1k):
    wl = wnl.lemmatize(w)
    ss = wn._synset_from_pos_and_offset('n',int(k[1:]))
    print ss, k, w
    direct_hypo = direct_hyponyms(ss)
    counter = Counter()
    countErr = 0
    v = captions[k]
    
    for cap in v:
        cap_lema = [wnl.lemmatize(w) for w in nltk.word_tokenize(cap)]
        isContains = False
        for w in cap_lema:
            if w in direct_hypo:
                counter[w] += 1
                isContains = True
        if isContains:
            countErr += 1
            
    print countErr, len(v), countErr*100.0/len(v)

Synset('serviceman.n.01') n10582746 man
2 1418 0.141043723554
Synset('woman.n.01') n10787470 woman
80 1405 5.69395017794
Synset('table.n.02') n04379243 table
39 1945 2.00514138817
Synset('street.n.01') n04334599 street
19 1297 1.46491904395
Synset('person.n.01') n00007846 person
906 1242 72.9468599034
Synset('plain.n.01') n09393605 field
0 1289 0.0
Synset('top.n.01') n08663860 top
0 59 0.0
Synset('tennis.n.01') n00482298 tennis
0 1837 0.0
Synset('battlefront.n.01') n08573674 front
0 13 0.0
Synset('room.n.01') n04105893 room
470 1326 35.4449472097
Synset('home_plate.n.01') n03528901 plate
0 1183 0.0
Synset('train.n.01') n04468005 train
0 1312 0.0
Synset('dog.n.01') n02084071 dog
16 1603 0.998128509046
Synset('water.n.06') n07935504 water
0 759 0.0
Synset('cat.n.01') n02121620 cat
0 1485 0.0
Synset('baseball.n.01') n00471613 baseball
563 1851 30.415991356
Synset('bathroom.n.01') n02807731 bathroom
0 1304 0.0
Synset('sign.n.02') n06793231 sign
0 1201 0.0
Synset('kitchen.n.01') n03619890 k

In [34]:
# Get full hypo
from collections import Counter
errPerWord = {}
for i,(k,w) in enumerate(top1k):
    wl = wnl.lemmatize(w)
    ss = wn._synset_from_pos_and_offset('n',int(k[1:]))
    print ss
    full_hypo = full_hyponyms(ss)
    counter = Counter()
    countErr = 0
    v = captions[k]
    for cap in v:
        cap_lema = [wnl.lemmatize(w) for w in nltk.word_tokenize(cap)]
        isContains = False
        for w in cap_lema:
            if w in full_hypo or w == wl:
                counter[w] += 1
                isContains = True
        if isContains:
            countErr += 1
            
    print countErr, len(v), countErr*100.0/len(v)
    print counter

Synset('serviceman.n.01')
721 1418 50.8462623413
Counter({u'man': 739, u'soldier': 13, u'officer': 2})
Synset('woman.n.01')
933 1405 66.4056939502
Counter({u'woman': 902, u'cat': 52, u'girl': 29, u'bird': 14, u'baby': 4})
Synset('table.n.02')
1075 1945 55.2699228792
Counter({u'table': 1055, u'bench': 367, u'desk': 26, u'counter': 12, u'stand': 1})
Synset('street.n.01')
609 1297 46.9545104086
Counter({u'street': 695, u'alley': 19})
Synset('person.n.01')
1201 1242 96.6988727858
Counter({u'man': 901, u'woman': 238, u'suit': 124, u'beard': 117, u'red': 90, u'dog': 75, u'front': 72, u'hand': 48, u'cat': 43, u'person': 33, u'boy': 32, u'head': 30, u'chair': 15, u'mouth': 15, u'bird': 13, u'bag': 12, u'bear': 11, u'girl': 9, u'child': 8, u'trick': 8, u'back': 7, u'computer': 7, u'controller': 7, u'face': 6, u'pink': 6, u'grass': 6, u'rock': 5, u'fence': 4, u'baby': 4, u'cow': 4, u'brick': 3, u'sheep': 2, u'image': 1, u'competition': 1, u'monitor': 1, u'sun': 1, u'skateboarder': 1, u'power': 1

In [42]:
# Full hyponyms with Depth
def full_hyponyms_with_depth(ss):
    full_hypo = {}
    q = [{'synset':ss, 'depth':0}]
    while len(q) > 0:
        a = q.pop(0)
        q += [{'synset': s, 'depth': a['depth']-1} 
             for s in a['synset'].hyponyms()]
        for name in a['synset'].lemma_names():
            full_hypo[name] = a['depth']
    return full_hypo

In [43]:
# Get full hypo with Depth
from collections import Counter
errPerWord = {}
for i,(k,w) in enumerate(top1k):
    wl = wnl.lemmatize(w)
    ss = wn._synset_from_pos_and_offset('n',int(k[1:]))
    print ss
    full_hypo = full_hyponyms_with_depth(ss)
    counter = Counter()
    countErr = 0
    v = captions[k]
    for cap in v:
        cap_lema = [wnl.lemmatize(w) for w in nltk.word_tokenize(cap)]
        isContains = False
        for w in cap_lema:
            if w in full_hypo or w == wl:
                counter[w] += 1
                isContains = True
        if isContains:
            countErr += 1
            
    print countErr, len(v), countErr*100.0/len(v)
    print counter

Synset('serviceman.n.01')
721 1418 50.8462623413
Counter({u'man': 739, u'soldier': 13, u'officer': 2})
Synset('woman.n.01')
933 1405 66.4056939502
Counter({u'woman': 902, u'cat': 52, u'girl': 29, u'bird': 14, u'baby': 4})
Synset('table.n.02')
1075 1945 55.2699228792
Counter({u'table': 1055, u'bench': 367, u'desk': 26, u'counter': 12, u'stand': 1})
Synset('street.n.01')
609 1297 46.9545104086
Counter({u'street': 695, u'alley': 19})
Synset('person.n.01')
1201 1242 96.6988727858
Counter({u'man': 901, u'woman': 238, u'suit': 124, u'beard': 117, u'red': 90, u'dog': 75, u'front': 72, u'hand': 48, u'cat': 43, u'person': 33, u'boy': 32, u'head': 30, u'chair': 15, u'mouth': 15, u'bird': 13, u'bag': 12, u'bear': 11, u'girl': 9, u'child': 8, u'trick': 8, u'back': 7, u'computer': 7, u'controller': 7, u'face': 6, u'pink': 6, u'grass': 6, u'rock': 5, u'fence': 4, u'baby': 4, u'cow': 4, u'brick': 3, u'sheep': 2, u'image': 1, u'competition': 1, u'monitor': 1, u'sun': 1, u'skateboarder': 1, u'power': 1

In [ ]:
start = datetime.now()
    
caption_file = 'data/annotations/captions_train2014.json'
image_dir = 'image/train2014_resized'
max_length = 15
word_count_threshold = 1
vgg_model_path = './data/imagenet-vgg-verydeep-19.mat'
batch_size = 50

print '1. Building Top 1K dictionary from Train dataset'

if not os.path.exists('./data/top1k.pkl'):
    train_dataset = _process_caption_data(caption_file=caption_file,
                                      image_dir=image_dir,
                                      max_length=max_length)
    word_to_idx = _build_vocab(annotations=train_dataset, threshold=word_count_threshold)
    save_pickle(word_to_idx, './data/word_to_idx.pkl')
    top1k = _build_top1k_vocab(train_dataset)
    save_pickle(top1k, './data/top1k.pkl')
else:
    top1k = load_pickle('./data/top1k.pkl')

In [ ]:
ss = wn._synset_from_pos_and_offset('n',9638875)

In [ ]:
ss.lemma_names()


# Let's build Top 1k using Part of Speech Tagger 

In [ ]:
caption_file = 'data/annotations/captions_train2014.json'
from nltk.tag.stanford import StanfordPOSTagger
from collections import Counter
import json
def get_top1k_noun(caption_file):
    counter = Counter()
    with open(caption_file) as f:
        caption_data = json.load(f)
    st = StanfordPOSTagger('stanford-postagger-2016-10-31/models/english-left3words-distsim.tagger',
                           path_to_jar='stanford-postagger-2016-10-31/stanford-postagger.jar')
    
    for anno in caption_data['annotations']:
        counter += Counter(st.tag(anno['caption'].split()))
        
    nouns = [c for c in counter.most_common(3000) if c[0][1] == 'NN']
    
    return nouns[:1000]
        

In [ ]:
top1k_noun =  get_top1k_noun(caption_file)

In [ ]:
import ipdb

In [ ]:
import pandas as pd


In [ ]:
tags = pd.read_table('captions.tag', names=['word','pos'])
nouns = tags[tags.pos == 'NN']
nouns['lemma'] = nouns.apply(lambda x: wn.morphy(x['word']), axis=1)
from collections import Counter
c = Counter(nouns['lemma'].values)


In [ ]:
frequentWords = c.most_common()

#Read imagenet synsets
with open('./data/imagenet.synsets','r') as f:
    synsets = f.readlines()
imagenet_synsets = {w.rstrip():True for w in synsets}

top1k = []

i = 0
top1k_dict = {}

# from IPython.core.debugger import Tracer
# Tracer()()
while len(top1k) < 1000:
    w = frequentWords[i][0]
    if w:
        j = 0
        wnid = 'a'
        while j<len(wn.synsets(w, pos=wn.NOUN)) and wnid not in imagenet_synsets:
            ss = wn.synset(frequentWords[i][0]+'.n.'+str(j+1).zfill(2))
            wnid = ss.pos() + str(ss.offset()).zfill(8)
            j += 1
        if wnid not in top1k_dict:
            top1k.append((wnid,w))
            top1k_dict[wnid] = w
    i += 1

In [ ]:
top1k

In [ ]:
wordnet_dict = pd.read_table('./data/words.txt',names=['wnid','words'])

In [ ]:
wordnet_dict.head()

In [ ]:
wordnet_dict['words'] = wordnet_dict.apply(lambda x: str(x['words']).split(','), axis=1)

In [ ]:
len(wordnet_dict)

In [ ]:
w

# Let's build a bar chart

In [65]:
?df.applymap
df = pd.DataFrame(top1k,columns=['wnid','word'])
df['captions'] = df.apply(lambda x: captions[x.wnid], axis=1)
df['num_captions'] = df.apply(lambda x: len(x.captions), axis=1)
df.head()

,wnid,word,captions,num_captions
0,n10582746,man,[a man in a suit and tie standing in a field ....,1418
1,n10787470,woman,"[two women are standing together with a tie .,...",1405
2,n04379243,table,[a living room with a wooden table and chairs ...,1945
3,n04334599,street,[a train traveling down tracks next to a fores...,1297
4,n00007846,person,"[a man and a woman are posing for a picture .,...",1242


In [82]:
def get_word_err_caps(word, lemma_captions):
    errCaps = []
    wl = wnl.lemmatize(word)
    for i, cap in enumerate(lemma_captions):
        if wl not in cap:
            errCaps.append(i)
    return errCaps

def lemmatize_captions(captions):
    lemma_captions = []
    for cap in captions:
        lemma_captions.append( map(wnl.lemmatize, nltk.word_tokenize(cap)) )
    return lemma_captions
    

In [74]:
df['lemmatized_captions'] = df.apply(lambda x: lemmatize_captions(x.captions), axis=1)

In [83]:
df['err_word_matching'] = df.apply(lambda x: get_word_err_caps(x.word, x.lemmatized_captions), axis=1)

In [84]:
df['err_word_rate'] = df.apply(lambda x: len(x.err_word_matching)*100.0/x.num_captions, axis=1)
df.head()


,wnid,word,captions,num_captions,err_word_matching,lemmatized_captions,err_word_rate
0,n10582746,man,[a man in a suit and tie standing in a field ....,1418,"[4, 6, 11, 14, 17, 18, 21, 25, 26, 28, 36, 37,...","[[a, man, in, a, suit, and, tie, standing, in,...",50.070522
1,n10787470,woman,"[two women are standing together with a tie .,...",1405,"[5, 6, 11, 15, 16, 17, 18, 19, 20, 21, 28, 31,...","[[two, woman, are, standing, together, with, a...",37.224199
2,n04379243,table,[a living room with a wooden table and chairs ...,1945,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 14, 18, 19, 2...","[[a, living, room, with, a, wooden, table, and...",62.673522
3,n04334599,street,[a train traveling down tracks next to a fores...,1297,"[0, 1, 3, 4, 6, 7, 9, 11, 12, 14, 19, 20, 22, ...","[[a, train, traveling, down, track, next, to, ...",54.433308
4,n00007846,person,"[a man and a woman are posing for a picture .,...",1242,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[a, man, and, a, woman, are, posing, for, a, ...",97.342995


In [86]:
data = [Bar(x=df.word,
            y=df.err_word_rate)]
py.iplot(data)

In [ ]:
def direct_hyponyms(synset): 
    direct_hypo = [] 
    for ss in synset.hyponyms():
        direct_hypo += ss.lemma_names()
    direct_hypo =  sorted(direct_hypo) 
    return direct_hypo
